In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics import roc_auc_score

# Add project root to path (where ensemble/ package is located)
project_root = Path.cwd().parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import refactored modules
from ensemble.config import EnsembleConfig
from ensemble.data import create_three_way_split, create_base_preprocessor
from ensemble.tracking import EnsembleDatabase, setup_logger
from ensemble.parallel import prepare_training_batch, train_batch_parallel
from ensemble.core import AcceptanceCriterion, DiversityScorer
from ensemble.stage2 import train_stage2_dnn, evaluate_ensemble, build_stage2_dnn
from ensemble.utils import compute_pipeline_hash

2025-12-11 23:37:02.190658: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765514222.218121 2313894 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765514222.226677 2313894 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Configuration

In [2]:
# Load configuration
config = EnsembleConfig()

# Training parameters
MAX_ITERATIONS = 1000
BATCH_SIZE = 20
N_WORKERS = 20
TIMEOUT_MINUTES = 60

# Paths
DATA_PATH = 'https://gperdrizet.github.io/FSA_devops/assets/data/unit3/diabetes_prediction_train.csv'
DB_PATH = Path.cwd().parent / 'data' / 'ensemble_training.db'
LOG_DIR = Path.cwd().parent / 'logs'
MODEL_DIR = Path.cwd().parent / 'models'

# Create run directory and log file path
run_id = datetime.now().strftime('run_%Y%m%d_%H%M%S')
run_dir = MODEL_DIR / run_id
run_dir.mkdir(parents=True, exist_ok=True)
LOG_DIR.mkdir(parents=True, exist_ok=True)
log_file = LOG_DIR / 'ensemble_training.log'

print(f'Run ID: {run_id}')
print(f'Run directory: {run_dir}')
print(f'Database: {DB_PATH}')
print(f'Log file: {log_file}')

Run ID: run_20251211_233707
Run directory: /mnt/arkk/kaggle/diabetes-prediction/models/run_20251211_233707
Database: /mnt/arkk/kaggle/diabetes-prediction/data/ensemble_training.db
Log file: /mnt/arkk/kaggle/diabetes-prediction/logs/ensemble_training.log


## Feature Definitions

In [3]:
# Define numerical features
numerical_features = [
    'age', 'physical_activity_minutes_per_week', 'diet_score',
    'sleep_hours_per_day', 'screen_time_hours_per_day', 'bmi',
    'waist_to_hip_ratio', 'systolic_bp', 'diastolic_bp', 'heart_rate',
    'cholesterol_total', 'hdl_cholesterol', 'ldl_cholesterol', 'triglycerides',
]

# Define nominal features (one-hot encoding)
nominal_features = [
    'gender', 'ethnicity', 'smoking_status', 'employment_status'
]

# Define ordinal features
ordinal_features = [
    'education_level', 'income_level', 'alcohol_consumption_per_week',
    'family_history_diabetes', 'hypertension_history', 'cardiovascular_history'
]

# Define ordinal categories in order
education_categories = ['No formal', 'Highschool', 'Graduate', 'Postgraduate']
income_categories = ['Low', 'Lower-Middle', 'Middle', 'Upper-Middle', 'High']
alcohol_categories = [1, 2, 3, 4, 5, 6, 7, 8, 9]
diabetes_history_categories = [0, 1]
hypertension_history_categories = [0, 1]
cardiovascular_history_categories = [0, 1]

ordinal_categories = [
    education_categories,
    income_categories,
    alcohol_categories,
    diabetes_history_categories,
    hypertension_history_categories,
    cardiovascular_history_categories
]

print(f'Numerical features: {len(numerical_features)}')
print(f'Nominal features: {len(nominal_features)}')
print(f'Ordinal features: {len(ordinal_features)}')

Numerical features: 14
Nominal features: 4
Ordinal features: 6


## Initialize Components

In [4]:
# Load data
print('Loading data...')
train_df = pd.read_csv(DATA_PATH)
train_df = train_df.drop(columns=['id'])
print(f'Loaded {len(train_df):,} samples')

# Create three-way split
print('\nCreating data splits...')
X_train_pool, X_val_s1, X_val_s2, y_train_pool, y_val_s1, y_val_s2 = create_three_way_split(
    train_df,
    'diagnosed_diabetes',
    random_state=config.random_state
)

print(f'Training pool: {len(X_train_pool):,} samples')
print(f'Stage 1 validation: {len(X_val_s1):,} samples')
print(f'Stage 2 validation: {len(X_val_s2):,} samples')

# Create base preprocessor
print('\nCreating preprocessor...')
base_preprocessor = create_base_preprocessor(
    numerical_features=numerical_features,
    ordinal_features=ordinal_features,
    nominal_features=nominal_features,
    ordinal_categories=ordinal_categories
)
print('✓ Preprocessor ready')

# Initialize database
print('\nInitializing database...')
database = EnsembleDatabase(db_path=DB_PATH)
database.reset()
database.initialize()

# Setup logger
logger = setup_logger(name='ensemble_training', log_file=log_file)
logger.info(f'Starting training run: {run_id}')

# Initialize trackers
current_ensemble = []
current_stage1_auc = 0.0
current_stage2_auc = 0.0
temperature = config.hill_climbing.base_temperature
best_auc = 0.0
plateau_counter = 0

print('\n✓ All components initialized')

Loading data...
Loaded 700,000 samples

Creating data splits...
Loaded 700,000 samples

Creating data splits...


[2025-12-11 23:37:11] INFO: Starting training run: run_20251211_233707


Training pool: 420,000 samples
Stage 1 validation: 244,999 samples
Stage 2 validation: 35,001 samples

Creating preprocessor...
✓ Preprocessor ready

Initializing database...
Deleted existing database: /mnt/arkk/kaggle/diabetes-prediction/data/ensemble_training.db
Database initialized at: /mnt/arkk/kaggle/diabetes-prediction/data/ensemble_training.db

✓ All components initialized


## Main Training Loop

In [ ]:
# Initialize acceptance criterion and diversity scorer
acceptance = AcceptanceCriterion(temperature=temperature, random_state=config.random_state)
diversity_scorer = DiversityScorer()

# Helper function to evaluate Stage 1 ensemble
def evaluate_stage1_ensemble(models, X, y):
    """Simple Stage 1 ensemble evaluation using averaging."""
    if not models:
        return 0.0
    
    predictions = []
    for model in models:
        try:
            pred_proba = model.predict_proba(X)[:, 1]
            predictions.append(pred_proba)
        except Exception as e:
            print(f"Warning: Model prediction failed: {e}")
            continue
    
    if not predictions:
        return 0.0
    
    # Average predictions
    avg_predictions = np.mean(predictions, axis=0)
    return roc_auc_score(y, avg_predictions)

print(f'Starting hill climbing with {BATCH_SIZE} workers...')
print(f'Target: {MAX_ITERATIONS} iterations')
print('=' * 70)

for batch_num in range(MAX_ITERATIONS // BATCH_SIZE):
    iteration_start = batch_num * BATCH_SIZE
    
    # Prepare batch
    logger.info(f'Batch {batch_num}: Preparing {BATCH_SIZE} candidates')
    
    batch_jobs = prepare_training_batch(
        iteration=iteration_start,
        batch_size=BATCH_SIZE,
        max_iterations=MAX_ITERATIONS,
        X_train_pool=X_train_pool,
        y_train_pool=y_train_pool,
        X_val_s1=X_val_s1,
        y_val_s1=y_val_s1,
        base_preprocessor=base_preprocessor,
        random_state=config.random_state,
        total_cpus=N_WORKERS,
        timeout_minutes=TIMEOUT_MINUTES,
        batch_num=batch_num
    )
    
    # Train batch in parallel
    results = train_batch_parallel(
        batch_jobs=batch_jobs,
        database=database,
        logger=logger,
        max_workers=N_WORKERS
    )
    
    # Process results
    successful = [r for r in results if r is not None]
    logger.info(f'Batch {batch_num}: {len(successful)}/{len(results)} completed')
    print(f'\n🔍 DEBUG: Batch {batch_num} - Total results: {len(results)}, Successful: {len(successful)}')
    
    if len(successful) == 0:
        print(f'⚠️  No successful results in batch {batch_num}, skipping to next batch')
        continue
    
    for result in successful:
        iteration = result['iteration']
        pipeline = result['pipeline']
        val_accuracy = result['val_accuracy']
        
        print(f'  Processing iteration {iteration}, classifier: {result.get("classifier_type", "unknown")}')
        
        # Compute pipeline hash
        pipeline_hash = compute_pipeline_hash(pipeline, result)
        
        # Evaluate candidate ensemble
        candidate_ensemble = current_ensemble + [pipeline]
        candidate_stage1_auc = evaluate_stage1_ensemble(
            candidate_ensemble, X_val_s1, y_val_s1
        )
        
        # Compute diversity
        if len(candidate_ensemble) > 1:
            predictions = np.array([p.predict_proba(X_val_s1)[:, 1] 
                                   for p in candidate_ensemble])
            diversity_score = diversity_scorer.score(predictions)
        else:
            diversity_score = 0.0
        
        # Acceptance decision
        accepted, reason = acceptance.should_accept(
            current_score=current_stage1_auc,
            candidate_score=candidate_stage1_auc
        )
        
        if accepted:
            current_ensemble = candidate_ensemble
            current_stage1_auc = candidate_stage1_auc
            
            if candidate_stage1_auc > best_auc:
                best_auc = candidate_stage1_auc
                plateau_counter = 0
            else:
                plateau_counter += 1
            
            logger.info(
                f'Iteration {iteration}: ACCEPTED - '
                f'Stage1 AUC={candidate_stage1_auc:.4f}, '
                f'Ensemble size={len(current_ensemble)}, '
                f'Diversity={diversity_score:.4f}'
            )
        else:
            plateau_counter += 1
        
        # Log to database
        database.insert_iteration({
            'timestamp': datetime.now().isoformat(),
            'iteration_num': iteration,
            'ensemble_id': f'ensemble_{iteration}',
            'stage1_val_auc': candidate_stage1_auc,
            'stage2_val_auc': current_stage2_auc,
            'diversity_score': diversity_score,
            'temperature': acceptance.temperature,
            'accepted': 1 if accepted else 0,
            'rejection_reason': reason if not accepted else None,
            'num_models': len(candidate_ensemble),
            'classifier_type': result['classifier_type'],
            'transformers_used': '',
            'use_pca': 0,
            'pca_components': None,
            'pipeline_hash': pipeline_hash,
            'training_memory_mb': result.get('memory_mb', 0.0),
            'stage2_memory_mb': 0.0,
            'training_time_sec': 0.0,
            'stage2_time_sec': 0.0,
            'timeout': 0,
            'stage2_tp': None,
            'stage2_fp': None,
            'stage2_tn': None,
            'stage2_fn': None
        })
        print(f'  ✓ Logged iteration {iteration} to database')
    
    # Update temperature using decay method
    acceptance.decay_temperature(config.hill_climbing.temperature_decay)
    
    # Status update
    print(f'\nBatch {batch_num} complete:')
    print(f'  Ensemble size: {len(current_ensemble)}')
    print(f'  Best Stage 1 AUC: {best_auc:.4f}')
    print(f'  Temperature: {acceptance.temperature:.6f}')
    print(f'  Plateau counter: {plateau_counter}')
    
    # Check for early stopping
    if plateau_counter >= config.hill_climbing.plateau_iterations:
        logger.info(f'Plateau detected after {plateau_counter} iterations. Stopping.')
        print(f'\n⚠ Plateau detected. Stopping early.')
        break

print('\n' + '=' * 70)
print('Training complete')
print(f'Final ensemble size: {len(current_ensemble)}')
print(f'Best Stage 1 AUC: {best_auc:.4f}')

[2025-12-11 23:37:11] INFO: Batch 0: Preparing 20 candidates


Starting hill climbing with 20 workers...
Target: 1000 iterations
Worker 0: Starting ridge (iteration 0, 119701 samples)
Worker 0: Starting ridge (iteration 0, 119701 samples)


## Save Results

In [ ]:
import joblib

# Save ensemble models
ensemble_dir = run_dir / 'ensemble_stage1_models'
ensemble_dir.mkdir(exist_ok=True)

for idx, model in enumerate(current_ensemble):
    model_path = ensemble_dir / f'model_{idx}.joblib'
    joblib.dump(model, model_path)

print(f'✓ Saved {len(current_ensemble)} models to {ensemble_dir}')

# Save metadata
metadata = {
    'run_id': run_id,
    'timestamp': datetime.now().isoformat(),
    'ensemble_size': len(current_ensemble),
    'best_stage1_auc': best_auc,
    'config': config.__dict__
}

import json
with open(run_dir / 'metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2, default=str)

print(f'Saved metadata to {run_dir / "metadata.json"}')